# Entrez physiquement dans le monde de Minecraft!

Comme projet de l'année en informatique, nous avons décider d'essayer de créer une **réalité virtuelle dans Minecraft** à l'aide d'un **Raspberry Pie**. Après de nombreuses heures de travailles, nous avons réussi à créer, avec des moyens très simples à créer une façon d'**incarner réeelement le personnage du monde de Minecraft** et de le faire bouger dans le jeu en bougeant dans la réalité. 

Nous avons présenté ce projet au **concours Bugnplay**, plus grand concours d'informatique de Suisse, ou nous avons été invité à présenter notre projet lors de la **finale du concours**. Le côté **original** et **amusant** de notre projet nous a permis d'y remporter deux prix, à savoir le **Best Classroom Project** et le **Senior Swiss Space Award**:

### Description de l'installation

Nous avons créer une installation plutôt complexe avec un Raspberry Pie, un écran, 2 souris, 1 batterie externe, 2 plaquette GPIO, 6 boutons, 3 mini-servos, une plaque en  bois à placer sous les pieds et surtout une armure que nous avons construit nous même.

#### Déplacement

Pour ce qui est du déplacement du personnage, nous avons créer une plaque sur laquelle nous avons fixé deux boutons. Lorsque l'on marche sur cette plaque, le personnage marche dans le monde de Minecraft. Le principe est  le suivant: lorsque l'un des deux boutons est enfoncé, la touche `w` destinée à faire avancer le joueur est simulée. Lorsque l'on appuie sur rien ou sur les deux boutons, rien ne se passe.

#### Construire et casser des briques

Afin de construire et casser des briques, nous avons posé sur notre armure 2 boutons au niveau des mains destinés à construire et à casser des briques. Chaque bouton est relié à un mini-servo qui sont placés de tel façon à cliquer respectivement sur le bouoton droit et le bouton gauche de la souris lorsque ils sont solicités. Lorsque l'on appuie sur ces boutons dans la réalité, les servos sont solicités de tel sorte à cliquer sur la souris pour créer ou détruire des bloques dans le monde de Minecraft. 

#### Rotation du personnage

Pour pouvoir tourner dans le monde de Minecraft en tournant dans la réalité, nous avons placé 2 boutons au niveau de la tête de telle façon à lorsque nous tournons la tête à droite (resp. à gauche), le bouton droite (resp. gauche) est appuyé. Ces boutons provoquent une rotation à droite (resp. à gauche) d'un troisième mini-servo. Celui-ci est attaché à une plaque qui bouge sous une souris. La souris detecte ddonc un mouvement vers la droite(resp. gauche) et fait tourner le personnage vers la droite.

### Code

#### Importation des modules nécessaires.

In [1]:
import time
import pyautogui
import RPi.GPIO as gpio
from time import sleep
from mcpi.minecraft import Minecraft

ImportError: No module named 'pyautogui'

#### Définition des boutons sur le GPIO

In [2]:
# Boutons pour avancer (plaque des pieds)
buttonL = 14
buttonR = 15

# Boutons pour créer et casser brique
buttonA = 23   # clique gauche souris, connecter à pwmL
buttonB = 24   # clique droit souris, connecter à pwmR

# Boutons pour la rotation du personnage, connectés à pwm
buttonC = 20
buttonD = 21

#### Position du joueur sur Minecraft

In [3]:
mc = Minecraft.create()
x, y ,z = mc.player.getPos()

NameError: name 'Minecraft' is not defined

#### Définition des fonctions servant à créer le monde idéal

In [4]:
def monde_plat():
    """Créer un monde plat pour faciliter déplacement."""    
    mc.setBlocks(x-300 ,y , z-300 ,x+300 ,y,z+300 ,2 )
    mc.setBlocks(x-300, y+1, z-300, x+300, y+300, z+300, 0)
     

def arbre(a, b, c):
    """Créer un arbre basé sur la position actuelle du joueur."""
    mc.setBlocks(x+a, y, z+c ,x+a ,y+b,z+c ,17)
    mc.setBlocks(x+(a-2), y+b , z+(c-2) ,x+(a+2) ,y+b,z+(c+2) ,18)
    mc.setBlocks(x+(a-1) ,y+(b+1) , z+(c-1) ,x+(a+1) ,y+(b+1),z+(c+1) ,18)
    mc.setBlocks(x+a ,y+(b+2) , z+c ,x+a ,y+(b+2),z+c ,18)

def foret():
    """Crée une forêt avec 12 arbres."""
    arbre(6, 4, 4)
    arbre(10, 3, 4)
    arbre(20,4,8)
    arbre(3,8,9)
    arbre(5,9,9)
    arbre(7,7,7)
    arbre(3,9,67)
    arbre(50,10,34)
    arbre(65,3,56)
    arbre(34,13,90)
    arbre(90,7,45)
    arbre(27,8,74)
    arbre(78,4,5)

#### Fonctions servant à initialiser servos et boutons

In [5]:
def init_boutons():
    """Initialisation des 6 boutons en pull-up."""
    gpio.setmode(gpio.BCM) # Mode du GPIO
    gpio.setup(buttonL, gpio.IN, pull_up_down=gpio.PUD_UP)
    gpio.setup(buttonR, gpio.IN, pull_up_down=gpio.PUD_UP)
    gpio.setup(buttonA, gpio.IN, pull_up_down=gpio.PUD_UP)    
    gpio.setup(buttonB, gpio.IN, pull_up_down=gpio.PUD_UP)
    gpio.setup(buttonC, gpio.IN, pull_up_down=gpio.PUD_UP)
    gpio.setup(buttonD, gpio.IN, pull_up_down=gpio.PUD_UP)
    
def init_servos():
    """Initialiser les servo-moteurs."""
    #Servo pour rotation du personnage, connecté à boutons C et D
    gpio.setup(2, gpio.OUT)
    pwm = gpio.PWM(2, 50)

    # Servo pour clique gauche, connecté à bouton A
    gpio.setup(18, gpio.OUT)
    pwmL = gpio.PWM(18, 50)

    # Servo pour clique droit, connecté à bouton B
    gpio.setup(13, gpio.OUT)
    pwmR = gpio.PWM(13, 50)

    ##Etat de base des servo
    pwmL.start(4)# Position  de base du servo pwmL
    pwmR.start(8)#  Etat de base du servo pwmR

    x = 7.5
    pwm.start(x)

#### Initialisation à l'état précédent des boutons.

In [6]:
L0 = False 
R0 = False
A0 = False 
B0 = False 
C0 = False
D0 = False

#### Fonctions qui initialise

In [8]:
init_boutons()
init_servos()
monde_plat()
foret()

NameError: name 'gpio' is not defined

Boucle

In [10]:
while True:
##  Code avancer buttons

    L = gpio.input(buttonL)
    R = gpio.input(buttonR)

    if not L and L0:
        pyautogui.keyDown('w')
    if L and not L0:
        pyautogui.keyUp('w')	
    L0 = L
    
    
    if not R and R0:
        pyautogui.keyDown('w')
    if R and not R0:
        pyautogui.keyUp('w')	
    R0 = R


## Code créer et casser brique avec servo pwmL et pwmR button A et button B

    A = gpio.input(buttonA)
    B = gpio.input(buttonB)

# clique gauche de la souris, casser une brique

    if not A and A0:
        pwmL.ChangeDutyCycle(2.5)
        time.sleep(0.5)
        pwmL.ChangeDutyCycle(4)
    if A and not A0:
        pass	    
    A0 = A
    
# clique droite de la souris, créer une brique

    if not B and B0:
        pwmR.ChangeDutyCycle(11)
        time.sleep(0.75)
        pwmR.ChangeDutyCycle(8)
    if B and not B0:
        pass    
    B0 = B
    

## code rotation personnage pwm et buttonC et buttonD

    C = gpio.input(buttonC)
    D = gpio.input(buttonD)

### tire le miservo vers 12.5

    if not C and C0 and x < 12.5:
        pwm.ChangeDutyCycle(x+2.5)
        x = x+2.5
    if C and not C0:
            pass 
    C0 = C

    # tire le mini servo vers 2.5
    
    if not D and D0 and x > 2.5:
        pwm.ChangeDutyCycle(x-2.5)
        x = x-2.5
    if D and not D0:
        pass		
    D0 = D

NameError: name 'gpio' is not defined